In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import io
from helper import *

/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.7'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
import mlflow
import mlflow.pytorch
from mlflow.tracking import MlflowClient

In [3]:
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as vutils

In [4]:
mlflow.set_experiment("Clasificador_Imagenes")

<Experiment: artifact_location='file:///Users/sofia/Desktop/ITBA/Redes/TP-Integrador-RN/mlruns/494564464208363776', creation_time=1749928798555, experiment_id='494564464208363776', last_update_time=1749928798555, lifecycle_stage='active', name='Clasificador_Imagenes', tags={'mlflow.sharedViewState.018699f78c20765e84c155638229fdc671d315a19ab0a5fc1ccf04a349360807': 'deflate;eJx1Ul1vIjEM/CtVntGpfeVti+iHCqfT0qJKqAKTmK6lbLKKHcq24r/XLLRLK92jxzPxxOMPwwjJVjfkBZMZGjMwMTlM1+0DtlrXKIks/1ltwS/B2pzAtqueVbA1ww14xoFhgSSPVKPqislESZ42aFvr8fv9wgptUVsOBBiFvzqLl4Gpo0M/x8QUQ6/w/qLMgVXD6NEKulH0uVZkuOj9SQIKPxz+13oDCWrFu3HnAIUmy5LpHc9Rl2ITs6yMOkxqZLxrIDh0ZvixPyL/KIRDvTgx7sg5DH09J6Y1eZJ2Ck0vO9Km6kK/eXNfzh6XV5fL8unvTKdvCd+msKNa3bjvFatsQixfA04ocWGtBqJrK7EL5kySGW9TzA26OfiMfB9Gleak25OUtV+Rw3HdSPsTfj1orvUEQvb+VP76OmSJJW4ScjUOsPYH/CT2cQ1+QgG7R0cxbOhVRWZX7IiPl8WCTXciAWd1jFIFZJ1+2cf8fCBPuxSPErPffwIf8PZb'}>

In [5]:
def log_classification_report(model, loader, writer, device, classes, step, prefix="val"):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    fig_cm, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
    disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
    ax.set_title(f'{prefix.title()} - Confusion Matrix')

    # Guardar localmente y subir a MLflow
    fig_path = f"confusion_matrix_{prefix}_epoch_{step}.png"
    fig_cm.savefig(fig_path)
    mlflow.log_artifact(fig_path)
    os.remove(fig_path)

    plot_to_tensorboard(fig_cm, writer, f"{prefix}/confusion_matrix", step)

    cls_report = classification_report(all_labels, all_preds, target_names=classes)
    writer.add_text(f"{prefix}/classification_report", f"<pre>{cls_report}</pre>", step)

    # También loguear texto del reporte
    with open(f"classification_report_{prefix}_epoch_{step}.txt", "w") as f:
        f.write(cls_report)
    mlflow.log_artifact(f.name)
    os.remove(f.name)


In [7]:
# Entrenamiento y validación
def evaluate(model, loader, writer, device, classes, epoch=None, prefix="val"):
    log_classification_report(model, loader, writer, device, classes, step=epoch , prefix="val")
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for i, (images, labels) in enumerate(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            loss_sum += loss.item()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Loguear imágenes del primer batch
            if i == 0 and epoch is not None:
                img_grid = vutils.make_grid(images[:8].cpu(), normalize=True)
                writer.add_image(f"{prefix}/images", img_grid, global_step=epoch)

    acc = 100.0 * correct / total
    avg_loss = loss_sum / len(loader)

    if epoch is not None:
        writer.add_scalar(f"{prefix}/loss", avg_loss, epoch)
        writer.add_scalar(f"{prefix}/accuracy", acc, epoch)

    return avg_loss, acc

In [8]:
# Paths
train_dir = r'skin-dataset-clasification/data/Split_smol/train/'
val_dir = r'skin-dataset-clasification/data/Split_smol/val/'

In [9]:
# Crear directorio de logs de tensorboard
log_dir = "runs/experimento_skin"
writer = SummaryWriter(log_dir=log_dir)

In [10]:
# Por nombre
experiment_name = "Clasificador_Imagenes"
experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id


client = MlflowClient()

# Opcional: Filtrá por condición, por ejemplo runs exitosos
MLP_models = client.search_runs(
    experiment_ids=[experiment_id],
    filter_string="parameter.model = 'MLPClassifier'", #and attributes.status = 'FINISHED'"
    order_by=["metrics.val_accuracy DESC"],  # o cualquier métrica que uses
    max_results=10
)

In [11]:
initializer=['uniform', 'He','xavier']

In [12]:
modelnbr=0
for i in range(3):
    for parameters in MLP_models:
        print(f"modelo número: {modelnbr}", end = "\r")
        modelnbr += 1
        hparams= {
            "model": ("MLPClassifier"),
            "input_size":  parameters.data.params['input_size'],
            "batch_size": parameters.data.params['batch_size'],
            "lr": parameters.data.params['lr'],
            "epochs": 200,
            "optimizer": "Adam" if parameters.data.params['optimizer']== "Adam" else "SGD" ,
            "HFlip": parameters.data.params['HFlip'],
            "VFlip": parameters.data.params['VFlip'],
            "RBContrast": parameters.data.params['RBContrast'],
            "loss_fn": "CrossEntropyLoss",
            "train_dir": train_dir,
            "val_dir": val_dir,
            "es_patience": 5,
            "dropout": parameters.data.params['dropout'],
            "BatchNorm": True,
            "Initializer": initializer[i]
        }
        train_transform = A.Compose([
            A.Resize(int(hparams["input_size"]), int(hparams["input_size"])),
            A.HorizontalFlip(p=hparams["HFlip"]),
            A.VerticalFlip(p=hparams["VFlip"]),
            A.RandomBrightnessContrast(p=hparams["RBContrast"]),
            A.Normalize(),
            ToTensorV2()
        ])
        val_test_transform = A.Compose([
            A.Resize(int(hparams["input_size"]), int(hparams["input_size"])),
            A.Normalize(),
            ToTensorV2()
        ])
        train_dataset = CustomImageDataset(train_dir, transform=train_transform)
        val_dataset   = CustomImageDataset(val_dir, transform=val_test_transform)
        batch_size = int(hparams["batch_size"])
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader   = DataLoader(val_dataset, batch_size=batch_size)
        device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
        num_classes = len(set(train_dataset.labels))
        model = MLPClassifier(num_classes=num_classes, input_size = int(hparams["input_size"])**2*3, BatchNorm=True, init_type=initializer[i]).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
        hparams["count_params"] = count_parameters(model)
        with mlflow.start_run():
            # Log hiperparámetros
            mlflow.log_params(hparams)
            best_val_acc = 0
            best_val_loss = 0
            best_train_acc = 0
            best_train_loss = 0
            best_epoch = 0
            for epoch in range(hparams["epochs"]):
                model.train()
                running_loss = 0.0
                correct, total = 0, 0
            
                for images, labels in train_loader:
                    images, labels = images.to(device), labels.to(device)
            
                    optimizer.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
            
                    running_loss += loss.item()
                    _, preds = torch.max(outputs, 1)
                    correct += (preds == labels).sum().item()
                    total += labels.size(0)
            
                train_loss = running_loss / len(train_loader)
                train_acc = 100.0 * correct / total
                val_loss, val_acc = evaluate(model, val_loader, writer, device,train_dataset.label_encoder.classes_,epoch=epoch, prefix="val")
            
            
                writer.add_scalar("train/loss", train_loss, epoch)
                writer.add_scalar("train/accuracy", train_acc, epoch)

                if epoch == 0:
                    for name, param in model.named_parameters():
                        writer.add_histogram(name, param, epoch)

                # Log en MLflow
                mlflow.log_metrics({
                    "train_loss": train_loss,
                    "train_accuracy": train_acc,
                    "val_loss": val_loss,
                    "val_accuracy": val_acc
                }, step=epoch)
                if val_acc > best_val_acc:
                    best_val_acc = val_acc
                    best_val_loss = val_loss
                    best_train_acc = train_acc
                    best_train_loss = train_loss
                    best_epoch = epoch
                    # Guardar modelo
                    torch.save(model.state_dict(), "mlp_model.pth")
                    #print("Modelo guardado como 'mlp_model.pth'")
                    mlflow.log_artifact("mlp_model.pth")
                    mlflow.pytorch.log_model(model, artifact_path="pytorch_model")
                elif epoch > best_epoch + int(hparams["es_patience"]):
                    #print("Early Stopping")
                    break
                    
            mlflow.log_metrics({
                    "train_loss": best_train_loss,
                    "train_accuracy": best_train_acc,
                    "val_loss": best_val_loss,
                    "val_accuracy": best_val_acc,
                    "best_epoch": best_epoch
                }, step=epoch+1)                                                
                                  

/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", 

/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", 

/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", 

/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", 

/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", 

...

/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", 

/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", 

2025/06/17 18:55:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Pre

2025/06/17 18:57:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/17 18:57:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/17 18:58:07 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/17 18:58:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
/opt/anaconda3/envs/TP_redes/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_

2025/06/17 18:59:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/17 18:59:13 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/17 18:59:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/17 18:59:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/17 18:59:56 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/17 19:00:14 